# Качество

В действительности мы не можем применять метрики *precision* и *recall*, так как мы имеем дело не с бинарной классификацией, а с многоклассовой. При этом число классов (то есть число возможных вершин) определяется для каждого преложения отдельно. В связи с этим для оценки качества нашего метода введём следующие метрики:

- Покрытие = число хоть как-то определённых связей во всех предложениях *(ТР + FP)* / число связей во всех предложений *(TP + FP + **TN + FN**)*
- "Точность" = число верно определённых связей во всех предложениях *(ТР)* / число связей, которые рассматривались в конструкциях *(TP + FP)*
- Тогда **TN + FN** -- число не рассматриваемых связей, то есть не различаем TN и FN, то есть не можем посчитать "recall".

*Осторожно!* Код считается довольно долго (часов 6-7). Раскомменьте последние ячейки для запуска.

In [12]:
import pandas as pd

In [13]:
import numpy as np

In [14]:
from collections import Counter

In [15]:
from tqdm import tqdm

In [26]:
def get_sents(test=True):
    list_of_conllus = ["./syntagrus/ru_syntagrus-ud-dev.conllu",
                       "./syntagrus/ru_syntagrus-ud-train.conllu",
                      "./syntagrus/ru_syntagrus-ud-test.conllu"]
    if test:
        list_of_conllus = ["./syntagrus/ru_syntagrus-ud-test.conllu"]

    sents = []
    for conllu_file in list_of_conllus:
        with open(conllu_file, encoding='utf-8') as f:
            lines = f.readlines()

        sents_one_file = []
        for line in lines:
            if line == "\n":
                s.append(("#", "_")) # маркер конца предложения
                sents_one_file.append(s)
            elif line.startswith('# text'): 
                s = [("#", "_")] # маркер начала предложения
            elif not(line.startswith('# sent_id')):
                word = line.split('\t')
                if '.' not in word[0]: # удалила строчки-подпорки для опущенных слов, т.к. они очень мешают дальше, а для нашей задачи пользы не несут
                    pos_word = word[3]
                    num_head = word[6]
                    if all([(num_head != "_"), (num_head != "0")]):
                        num_head = str(int(num_head) + 1)
                    s.append((pos_word, num_head)) # добавляем чр, номер хоста (номер слова, словоформу -- word[0], word[1])
        
        sents.extend(sents_one_file)

    return sents

In [27]:
def get_n_grams(n, sent): 
    n_grams = []
    for i in range(n, len(sent) + 1):
        slice_ = sent[i - n : i]
        gram = []
        for word in slice_:
            pos_word = word[0]
            num_head = word[1]
            if num_head == '_' or int(num_head) not in range(i + 1 - n, i + 1):
                num_head = '_'
            else:
                num_head = int(num_head) - (i - n)
            gram.extend([pos_word, str(num_head)])
        n_grams.append(tuple(gram))
    return n_grams

In [28]:
# Правильный порядок для записи
def right_col_order(n):
    right_order = []
    for pos, host in zip(generate_columns(n, "POS"), generate_columns(n, "#host")):
        right_order.extend([pos, host])
    return right_order

In [29]:
# Генерируем колонки по шаблону
def generate_columns(n, name):
    columns = [name+str(i) for i in range(1, n+1)]
    return columns

In [30]:
def got(i, row):
    if row[f"#host{i}_y"] == "0":
        return "0"
    elif row[f"#host{i}_x"] == row[f"#host{i}_y"]:
        # -1 -- сдвиг по списку
        return str(int(row["index"]) + i - 1)
    else:
        return str(int(row["index"]) + i - 1) + "*"

In [31]:
def count_T_F(test=True):
    all_sents = get_sents(test=test)
    if test:
        path = "./outcome files/dev_train/all_start_finish/"
    else:
        path = "./outcome files/dev_train_test/"
    count = {
             3: {"True": 0, "False": 0}, 
             4: {"True": 0, "False": 0},
             5: {"True": 0, "False": 0},
             6: {"True": 0, "False": 0},
             "all_n": {"True": 0, "False": 0},
             "all": 0
            }
    for sent in tqdm(all_sents):
        n_elem = len(sent)
        all_sent_links = ["*"] * n_elem
        count["all"] += (n_elem - 3)

        for n in range(3, 7):
            sent_links = ["*"] * n_elem
            df = pd.read_csv(path + f"all_cool_{n}_grouped.csv", sep=",", low_memory=False)
            df = df[df["total_entries"] > 50]
            sent_n = get_n_grams(n, sent)
            df_sent = pd.DataFrame(sent_n, columns=right_col_order(n))
            df_sent.reset_index(level=0, inplace=True)
            df_m = pd.merge(df_sent, df, on=generate_columns(n, "POS")).astype(str)

            if not df_m.empty:
                col_np = []
                for i in range(1, n+1):
                    df_m[f"got_{i}"] = df_m.apply(lambda row: got(i, row), axis=1)
                    col_np.append(f"got_{i}")
                nump = df_m[col_np].to_numpy()
                nums_word = np.unique(nump, return_counts=True)[0][1:]
                
                # Расставляем теги в предложении при анализе только этой ширины n-грамм
                for i in nums_word:
                    if i[-1] == "*":
                        sent_links[int(i[:-1])] = "False"
                    else:
                        sent_links[int(i)] = "True"
                        
                # Добавляем к счётчику данные с этой ширины n-грамм
                count_sent_n = Counter(sent_links)
                count[n]["True"] += count_sent_n["True"]
                count[n]["False"] += count_sent_n["False"]
                
                # Обновляем итоговые разбор предложения
                for i in nums_word:
                    if i[-1] == "*":
                        all_sent_links[int(i[:-1])] = "False"
                    else:
                        all_sent_links[int(i)] = "True"
                        
        # Добавляем к общему счетчику
        count_sent = Counter(all_sent_links)
        count["all_n"]["True"] += count_sent["True"]
        count["all_n"]["False"] += count_sent["False"]
    return count

In [32]:
def metrics(n, count):
    precision = count[n]["True"] / (count[n]["True"] + count[n]["False"])
    cover = (count[n]["True"] + count[n]["False"]) / count["all"]
    return precision, cover

In [33]:
def write(text, test=True):
    if test:
        path = "./outcome files/dev_train/"
    else:
        path = "./outcome files/dev_train_test/"
    with open(path + "quality.txt", "a", encoding="utf-8") as f:
        f.write(text)

In [34]:
def run_all(test=False):
    count = count_T_F(test=test)
    
    sum_true = 0
    sum_false = 0
    for n in range(3, 7):
        precision, cover = metrics(n, count)
        sum_true += count[n]["True"]
        sum_false += count[n]["False"]

        text = f"Для {n}-грамм:\nточность {precision}\nпокрытие {cover}\n\n"
        write(text, test=test)

    total_prec = sum_true / (sum_true + sum_false)
    total_cov = (sum_true + sum_false) / count["all"]
    total_text = f"Всего:\nточность {total_prec}\nпокрытие {total_cov}\n\n"
    write(total_text, test=test)

In [2]:
# На test
# run_all(test=True)

In [1]:
# # На всей коллекции
# run_all(test=False)